In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from LR import ar1_train
from MA import EMA_train
from KF import kf_train
from SVM import svm_train
#from PP import PP_train   # this model is too slow

import logging
logging.getLogger().setLevel(logging.ERROR)

import yfinance as yf
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
import statsmodels as smt

In [3]:
start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
visa = yf.download('V',start_date ,end_date,index=False)

[*********************100%***********************]  1 of 1 completed


In [9]:
visa.reset_index(inplace=True)

In [11]:
start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
JP = yf.download('JPM',start_date ,end_date,index=False)

start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
Master_card = yf.download('MA',start_date ,end_date,index=False)

start_date = datetime(2019,3,1)
end_date = datetime(2020,3,31)
Bofa = yf.download('BAC',start_date ,end_date,index=False)

df_vjp = visa.merge(JP, how='left', on='Date')
df_vjp = df_vjp.rename(columns=lambda x: x.replace('_x', ''))
df_vjp = df_vjp[['Date','Adj Close','Adj Close_y']]
df_vjp.rename(columns={'Adj Close':'Adj_Close_visa','Adj Close_y':'Adj_Close_JP'},inplace=True)
df_vjp
df_vmc = visa.merge(Master_card, how='left', on='Date')
df_vmc = df_vmc.rename(columns=lambda x: x.replace('_x', ''))
df_vmc = df_vmc[['Date','Adj Close','Adj Close_y']]
df_vmc.rename(columns={'Adj Close':'Adj_Close_visa','Adj Close_y':'Adj_Close_Master'},inplace=True)
dfdf_ff_vba = visa.merge(Bofa, how='left', on='Date')
df_vba = df_vba.rename(columns=lambda x: x.replace('_x', ''))
df_vba = df_vba[['Date','Adj Close','Adj Close_y']]
df_vba.rename(columns={'Adj Close':'Adj_Close_visa','Adj Close_y':'Adj_Close_BOfA'},inplace=True)
df_vba
df_final = df_vmc.merge(df_vjp, how='left', on=['Date','Adj_Close_visa'])
df_final = df_final.merge(df_vba, how='left', on=['Date','Adj_Close_visa'])
df_final



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Adj_Close_visa,Adj_Close_Master,Adj_Close_JP,Adj_Close_BOfA
0,2019-03-01,146.724609,223.466248,94.824356,27.388266
1,2019-03-04,145.242355,222.522217,94.606438,27.126625
2,2019-03-05,145.232529,222.453384,94.533783,27.173344
3,2019-03-06,145.095093,220.732513,94.179665,26.949080
4,2019-03-07,144.133087,218.706802,93.498642,26.678089
...,...,...,...,...,...
268,2020-03-24,152.619751,234.327652,82.663330,20.123720
269,2020-03-25,159.780106,253.513153,85.748131,20.190702
270,2020-03-26,166.792358,260.135590,91.721436,21.740889
271,2020-03-27,159.562836,244.785233,85.187256,20.669159


In [14]:
df_final.set_index(['Date'], inplace=True)

In [16]:
df_final = df_final.diff()

In [17]:
df_final = df_final[1:]

In [18]:
df_final

,Adj_Close_visa,Adj_Close_Master,Adj_Close_JP,Adj_Close_BOfA
Date,,,,
2019-03-04,-1.482254,-0.944031,-0.217918,-0.261641
2019-03-05,-0.009827,-0.068832,-0.072655,0.046719
2019-03-06,-0.137436,-1.720871,-0.354118,-0.224264
2019-03-07,-0.962006,-2.025711,-0.681023,-0.270990
2019-03-08,0.510483,-0.531021,0.036331,0.093449
...,...,...,...,...
2020-03-24,18.557724,33.379364,8.787003,2.822870
2020-03-25,7.160355,19.185501,3.084801,0.066982
2020-03-26,7.012253,6.622437,5.973305,1.550186


In [20]:
df_ff = pd.read_csv('F-F_Research_Data_Factors_daily.CSV',skiprows=4)

In [22]:
df_ff.rename(columns = {'Unnamed: 0':'Date'},inplace=True)
df_ff=df_ff[:-1]
df_ff['Date']=pd.to_datetime(df_ff['Date'])

/home/pramod/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [27]:
df_ff.set_index(['Date'], inplace=True)
visa.set_index(['Date'], inplace=True)

In [33]:
result = visa.merge(df_ff, how='left', on='Date')

In [35]:
result = result.merge(df_final, how='left', on='Date')

In [37]:
result = result[1:]

In [38]:
result

,Open,High,Low,Close,Adj Close,Volume,Mkt-RF,SMB,HML,RF,Adj_Close_visa,Adj_Close_Master,Adj_Close_JP,Adj_Close_BOfA
Date,,,,,,,,,,,,,,
2019-03-04,150.070007,150.190002,146.949997,147.960007,145.242355,6979700,-0.52,-0.39,0.39,0.009,-1.482254,-0.944031,-0.217918,-0.261641
2019-03-05,148.000000,149.289993,147.830002,147.949997,145.232529,9678600,-0.17,-0.31,-0.23,0.009,-0.009827,-0.068832,-0.072655,0.046719
2019-03-06,148.000000,148.520004,147.419998,147.809998,145.095093,8186400,-0.84,-1.27,0.05,0.009,-0.137436,-1.720871,-0.354118,-0.224264
2019-03-07,147.259995,147.809998,146.039993,146.830002,144.133087,8761700,-0.82,-0.07,-0.34,0.009,-0.962006,-2.025711,-0.681023,-0.270990
2019-03-08,145.440002,147.479996,144.500000,147.350006,144.643570,5168100,-0.22,0.06,-0.14,0.009,0.510483,-0.531021,0.036331,0.093449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-24,144.839996,155.250000,143.580002,154.529999,152.619751,24488300,9.34,-1.27,2.43,0.006,18.557724,33.379364,8.787003,2.822870
2020-03-25,159.320007,168.179993,153.580002,161.779999,159.780106,20619600,1.18,-0.70,1.87,0.006,7.160355,19.185501,3.084801,0.066982
2020-03-26,162.080002,169.529999,161.789993,168.880005,166.792358,17062900,6.02,-0.75,1.17,0.006,7.012253,6.622437,5.973305,1.550186
